In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
from pandas.tseries.offsets import BDay
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 5, 12), datetime.date(2023, 5, 11))

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
today, yesterday

(datetime.date(2023, 3, 19), datetime.date(2023, 3, 17))

In [3]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

In [4]:
sql = """
SELECT *
FROM dividend
LIMIT 1"""
struct = pd.read_sql(sql, const)
struct.dtypes

NAME         object
Q4          float64
Q3          float64
Q2          float64
Q1          float64
DIVIDEND    float64
PRICE       float64
PERCENT     float64
SHARES      float64
XDATE        object
PAIDDATE     object
KIND         object
ACTUAL        int64
dtype: object

### New dividend stock

In [5]:
name = 'TTB'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [6]:
sqlIns = """
INSERT INTO dividend
VALUES('TTB',.053,0,0,0,.053,0,0,000,'2023-04-18','2023-05-03','',0)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [5]:
def update_dividend1(q4,XDATE,PAIDDATE,actual,name):
    sql = "UPDATE dividend SET q4 = %s, dividend = q1+q2+q3+q4, XDATE = '%s', PAIDDATE = '%s', actual = %s WHERE name = '%s'"
    sql = sql % (q4,XDATE, PAIDDATE, actual,name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [6]:
name = 'KCE'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,KCE,1.0000,0.0000,1.0000,0.0000,2.0000,"10,000",2023-03-21,2023-05-09,,0


In [7]:
q4 = 0.6
XDATE = '2023-03-20'
PAIDDATE = '2023-05-12'
actual = 1

update_dividend1(q4,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [ ]:
def update_dividend2(shares,q4,actual,name):
    sql = "UPDATE dividend SET shares = %s, q4 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q4, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [ ]:
name = 'DTAC'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

In [ ]:
shares = 0000
q4 = 1.05
actual = 0

In [ ]:
update_dividend2(shares,q4,actual,name)

### Toggle actual status

In [2]:
name = 'SENA'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [ ]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("LPF")
'''
rp = const.execute(sqlDel)
rp.rowcount

### Start of output to cloud

In [5]:
sql = """
SELECT Y.NAME AS name,  Q4 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q4 > 0 
AND P.date = '%s'
ORDER BY name
"""
sql = sql % yesterday 
print(sql)


SELECT Y.NAME AS name,  Q4 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q4 > 0 
AND P.date = '2023-03-09'
ORDER BY name



In [6]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [7]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,ASK,1.4400,"4,500","6,480.00","5,832.00",2023-02-28,2023-04-27,"139,500.00",4.18%,0
1,ASP,0.1300,"30,000","3,900.00","3,510.00",2023-03-08,2023-05-17,"114,000.00",3.08%,0
2,BANPU,0.7500,"18,000","13,500.00","12,150.00",2023-04-10,2023-04-28,"216,000.00",5.62%,1
3,BCH,0.4000,"15,000","6,000.00","5,400.00",2023-05-09,2023-05-19,"321,900.00",1.68%,1
4,CPNREIT,0.0607,"55,000","3,338.50","3,004.65",2023-02-28,2023-03-16,"990,000.00",0.30%,0
5,DIF,0.2535,"30,000","7,605.00","6,844.50",2023-02-10,2023-03-07,"441,000.00",1.55%,0
6,GVREIT,0.1980,"40,000","7,920.00","7,128.00",2023-02-27,2023-03-10,"356,000.00",2.00%,0
7,IVL,0.4000,"7,200","2,880.00","2,592.00",2023-05-03,2023-05-19,"288,000.00",0.90%,1
8,JASIF,0.2300,"130,000","29,900.00","26,910.00",2023-03-02,2023-03-20,"1,300,000.00",2.07%,0
9,JMART,0.6600,"4,800","3,168.00","2,851.20",2023-04-11,2023-05-03,"178,800.00",1.59%,1


In [8]:
dividend.net.sum()

241918.65000000002

In [10]:
file_name = 'dividend-q4.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name
one_file = one_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_csv(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(osd_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(one_file, index=False)

### End of output to cloud

In [ ]:
sqlUpd = """
UPDATE dividend
SET xdate = DATE_ADD(xdate, INTERVAL 1 YEAR),
    paiddate = DATE_ADD(paiddate, INTERVAL 1 YEAR),
    actual = 0
WHERE Q4 > 0"""
rp = const.execute(sqlUpd)
rp.rowcount